# Запись

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

In [ ]:
data = [("Анна", 25), ("Иван", 30), ("Мария", 28), ("Осталый", 38)]
df = spark.createDataFrame(data, ["name", "age"])

# PostgreSQL

In [ ]:
# Базовый

(df.write.format("jdbc")
         .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
         .option("driver", "org.postgresql.Driver")
         .option("user", "airflow")
         .option("password", "airflow")
         .option("dbtable", "отсталые")
         .mode("overwrite") # перезаписать таблицу
       # .mode("append")    # добавить данные
       # .mode("ignore")    # пропустить если таблица существует
       # .mode("error")     # ошибка если таблица существует (по умолчанию)
         .save())

In [ ]:
# Расширенный

(df.write.format("jdbc")
         .option("batchsize", 10000)                  # размер батча
         .option("isolationLevel", "READ_COMMITTED")  # уровень изоляции
         .option("truncate", "true")                  # truncate вместо drop при overwrite
         .option("createTableColumnTypes", "name VARCHAR(255), age INT") # типы колонок
)

In [ ]:
# Параллельная запись с партиционированием

(df.write.format("jdbc")
         .option("numPartitions", 10)         # количество партиций
         .option("partitionColumn", "id")     # колонка для партиционирования
         .option("lowerBound", "1")           # минимальное значение
         .option("upperBound", "1000000")     # максимальное значение
)

In [ ]:
# Через функцию

def spark_to_postgres(df, table_name, mode="append", **options):
    
    base_options = {
                    "url": "jdbc:postgresql://postgres-db:5432/learn_base",
                    "driver": "org.postgresql.Driver",
                    "user": "airflow", 
                    "password": "airflow",
                   }

    base_options.update(options)
    
    (df.write
       .format("jdbc")
       .option("dbtable", table_name)
       .options(**base_options)
       .mode(mode)
       .save())

# Использование
spark_to_postgres(df, "отсталые", "overwrite", batchsize=20000, truncate="true")

In [ ]:
# Короткий

df.write.jdbc(
              url="jdbc:postgresql://postgres-db:5432/learn_base",
              table="отсталые",
              properties={
                          "user":     "airflow",
                          "password": "airflow",
                          "driver":   "org.postgresql.Driver"
                         },
              mode="overwrite"
              )

# CSV

In [ ]:
(df.write.format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("quote", "\"")
    .option("escape", "\"")
    .option("encoding", "UTF-8")
    .mode("overwrite")
    .save("s3a://my-bucket/data/csv-output"))

In [ ]:
(df.write.format("csv")
    .option("header", "true")              # Заголовки
    .option("delimiter", ",")              # Разделитель
    .option("quote", "\"")                 # Символ quoting
    .option("escape", "\"")                # Символ экранирования
    .option("encoding", "UTF-8")           # Кодировка
    .option("nullValue", "NULL")           # Замена null
    .option("dateFormat", "yyyy-MM-dd")    # Формат дат
    .option("compression", "gzip")         # Сжатие
    .mode("overwrite")                     # Режим записи
    .save("s3a://my-bucket/data/output"))  # Путь назначения

Подробно разберем каждую опцию записи CSV в этом формате:

### Базовый синтаксис
```python
(df.write.format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("quote", "\"")
    .option("escape", "\"")
    .option("encoding", "UTF-8")
    .mode("overwrite")
    .save("s3a://my-bucket/data/csv-output"))
```

---

### **1. `.format("csv")`**
**Назначение**: Указывает формат записи
```python
.format("csv")  # Запись в CSV формате
```
**Альтернативы**: `"parquet"`, `"json"`, `"orc"`, `"avro"`

---

### **2. `.option("header", "true")`**
**Назначение**: Включает запись заголовков столбцов
```python
.option("header", "true")   # ✅ Записать названия колонок
.option("header", "false")  # ❌ Без заголовков (по умолчанию)
```

**Пример данных:**
```
id,name,age
1,John,25
2,Jane,30
```

---

### **3. `.option("delimiter", ",")`**
**Назначение**: Задает разделитель полей
```python
.option("delimiter", ",")    # Стандартный CSV
.option("delimiter", ";")    # Европейский CSV
.option("delimiter", "|")    # Pipe-separated
.option("delimiter", "\t")   # TSV (табуляция)
```

**Результат с разными разделителями:**
```csv
# delimiter = ","
1,John,25

# delimiter = "|"
1|John|25
```

---

### **4. `.option("quote", "\"")`**
**Назначение**: Символ для обрамления полей
```python
.option("quote", "\"")    # Двойные кавычки (стандарт)
.option("quote", "'")     # Одинарные кавычки
.option("quote", "")      # Без quoting (опасно!)
```

**Когда используется:**
```csv
# Без quote
1,John Doe,25"  # ❌ Проблема с кавычкой в данных

# С quote
1,"John Doe","25""",Engineer  # ✅ Экранирование работает
```

---

### **5. `.option("escape", "\"")`**
**Назначение**: Символ экранирования специальных символов
```python
.option("escape", "\"")    # Экранирование кавычкой
.option("escape", "\\")    # Экранирование обратным слэшем
```

**Пример экранирования:**
```csv
# Данные: He said "Hello"
"He said ""Hello"""     # escape = "\""
"He said \"Hello\""     # escape = "\\"
```

---

### **6. `.option("encoding", "UTF-8")`**
**Назначение**: Кодировка файла
```python
.option("encoding", "UTF-8")      # Unicode (рекомендуется)
.option("encoding", "ISO-8859-1") # Latin-1
.option("encoding", "CP1251")     # Windows Cyrillic
```

---

### **7. `.mode("overwrite")`**
**Назначение**: Режим записи при существующем файле
```python
.mode("overwrite")   # 📝 Перезаписать существующие данные
.mode("append")      # ➕ Добавить к существующим
.mode("ignore")      # ⏭️ Пропустить если файл существует
.mode("error")       # ❌ Ошибка если файл существует (по умолчанию)
```

---

### **Дополнительные важные опции**

### **Управление NULL значениями**
```python
.option("nullValue", "NULL")     # Заменяет null на "NULL"
.option("nullValue", "")         # Заменяет null на пустую строку
.option("nullValue", "\\N")      # Как в MySQL
```

### **Форматы дат и времени**
```python
.option("dateFormat", "yyyy-MM-dd")          # 2024-01-15
.option("timestampFormat", "yyyy-MM-dd HH:mm:ss")  # 2024-01-15 14:30:00
```

### **Сжатие**
```python
.option("compression", "gzip")    # .csv.gz
.option("compression", "none")    # Без сжатия
```

---

## **Полный пример с всеми опциями**
```python
(df.write.format("csv")
    .option("header", "true")              # Заголовки
    .option("delimiter", ",")              # Разделитель
    .option("quote", "\"")                 # Символ quoting
    .option("escape", "\"")                # Символ экранирования
    .option("encoding", "UTF-8")           # Кодировка
    .option("nullValue", "NULL")           # Замена null
    .option("dateFormat", "yyyy-MM-dd")    # Формат дат
    .option("compression", "gzip")         # Сжатие
    .mode("overwrite")                     # Режим записи
    .save("s3a://my-bucket/data/output"))  # Путь назначения
```

---

## **Результат записи**
Для DataFrame:
```python
+---+----------+---+
| id|      name|age|
+---+----------+---+
|  1|  John Doe| 25|
|  2|Jane Smith| 30|
|  3|      NULL| 35|
+---+----------+---+
```

Будет записан CSV:
```csv
id,name,age
1,"John Doe",25
2,"Jane Smith",30
3,NULL,35
```

# Parquet

## **Запись Parquet в MinIO**

```python
(df.write.format("parquet")
    .option("compression", "snappy")
    .option("parquet.block.size", 134217728)
    .option("parquet.page.size", 1048576)
    .option("parquet.dictionary.enabled", "true")
    .option("parquet.bloom.filter.enabled", "true")
    .mode("overwrite")
    .save("s3a://my-bucket/data/parquet-output"))
```

---

## **1. `.format("parquet")`**
**Назначение**: Указывает формат Parquet
```python
.format("parquet")  # Запись в колоночном формате Parquet
```

---

## **2. `.option("compression", "snappy")`**
**Назначение**: Алгоритм сжатия данных
```python
.option("compression", "snappy")    # 🚀 Быстрое сжатие (рекомендуется)
.option("compression", "gzip")      # 📦 Высокое сжатие (медленнее)
.option("compression", "lzo")       # ⚡ Очень быстрое (меньше сжатие)
.option("compression", "none")      # 🔓 Без сжатия
.option("compression", "zstd")      # 🎯 Баланс скорости/сжатия (новое)
```

**Сравнение:**
- **Snappy**: Скорость + разумное сжатие
- **Gzip**: Медленнее, но лучшее сжатие
- **None**: Для часто изменяемых данных

---

## **3. `.option("parquet.block.size", 134217728)`**
**Назначение**: Размер блока данных в байтах
```python
.option("parquet.block.size", 134217728)  # 128 MB (по умолчанию)
.option("parquet.block.size", 268435456)  # 256 MB (для больших данных)
.option("parquet.block.size", 67108864)   # 64 MB (для маленьких файлов)
```

**Зачем нужно:**
- Блок - минимальная единица чтения
- Большие блоки → лучшее сжатие
- Меньшие блокы → лучше параллелизм

---

## **4. `.option("parquet.page.size", 1048576)`**
**Назначение**: Размер страницы в байтах
```python
.option("parquet.page.size", 1048576)     # 1 MB (по умолчанию)
.option("parquet.page.size", 524288)      # 512 KB
.option("parquet.page.size", 2097152)     # 2 MB
```

**Зачем нужно:**
- Страница - минимальная единица доступа
- Меньший размер → точечное чтение
- Больший размер → лучшее сжатие

---

## **5. `.option("parquet.dictionary.enabled", "true")`**
**Назначение**: Включить словарное кодирование
```python
.option("parquet.dictionary.enabled", "true")   # ✅ Включить (рекомендуется)
.option("parquet.dictionary.enabled", "false")  # ❌ Выключить
```

**Преимущества:**
- Эффективно для повторяющихся значений
- Уменьшает размер данных
- Особенно полезно для строковых колонок

---

## **6. `.option("parquet.bloom.filter.enabled", "true")`**
**Назначение**: Включить Bloom filter для быстрого поиска
```python
.option("parquet.bloom.filter.enabled", "true")    # ✅ Включить
.option("parquet.bloom.filter.enabled", "false")   # ❌ Выключить
```

**Зачем нужно:**
- Ускоряет предикатный pushdown
- Эффективно для поиска по конкретным значениям
- Особенно полезно для колонок с высоким кардиналитетом

---

## **7. `.mode("overwrite")`**
**Назначение**: Режим записи
```python
.mode("overwrite")   # 📝 Перезаписать существующие данные
.mode("append")      # ➕ Добавить к существующим
.mode("ignore")      # ⏭️ Пропустить если существует
.mode("error")       # ❌ Ошибка если существует
```

---

## **Дополнительные важные опции**

### **Настройка производительности**
```python
.option("parquet.writer.version", "v2")           # Версия формата
.option("parquet.enable.dictionary", "true")      # Словарное кодирование
.option("parquet.dictionary.page.size", 1048576)  # Размер словарной страницы
```

### **Схема данных**
```python
.option("mergeSchema", "true")                    # Объединять схемы при append
.option("parquet.strings.signed-min-max.enabled", "true")  # Для строковых статистик
```

### **Партиционирование**
```python
.partitionBy("year", "month")  # 🗂️ Партиционирование по колонкам
```

---

## **Полный пример с партиционированием**

```python
(df.write.format("parquet")
    .option("compression", "snappy")
    .option("parquet.block.size", 134217728)
    .option("parquet.page.size", 1048576)
    .option("parquet.dictionary.enabled", "true")
    .option("parquet.bloom.filter.enabled", "true")
    .option("parquet.writer.version", "v2")
    .option("mergeSchema", "true")
    .partitionBy("year", "month")           # Партиционирование
    .mode("overwrite")
    .save("s3a://my-bucket/data/partitioned-parquet/"))
```

---

## **Структура результата в MinIO**
```
s3a://my-bucket/data/parquet-output/
├── part-00000-xxx.snappy.parquet
├── part-00001-xxx.snappy.parquet
└── _SUCCESS

s3a://my-bucket/data/partitioned-parquet/
├── year=2024/
│   ├── month=01/
│   │   └── part-00000-xxx.snappy.parquet
│   └── month=02/
│       └── part-00000-xxx.snappy.parquet
└── year=2023/
    └── month=12/
        └── part-00000-xxx.snappy.parquet
```

---

## **Преимущества Parquet в MinIO**
- **✅ Колоночный формат** - эффективные запросы
- **✅ Сжатие** - экономия хранилища
- **✅ Предикатный pushdown** - фильтрация на чтении
- **✅ Schema evolution** - эволюция схемы данных
- **✅ Совместимость** - работа с различными инструментами